# CS235F25 HW 2

**Dataset:** Airbnb

# Imports

In [772]:
# [INSERT CODE TO IMPORT NECESSARY PYTHON PACKAGES HERE]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import ast
import warnings
warnings.filterwarnings('ignore')

# 0. Load Data / Data Exploration

In [773]:
trainDF = pd.read_csv('train.csv')
testDF = pd.read_csv('test.csv')

In [774]:
# Inspected the data type of each feature
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26830 entries, 0 to 26829
Data columns (total 65 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                26830 non-null  int64  
 1   name                              26823 non-null  object 
 2   summary                           25821 non-null  object 
 3   space                             18483 non-null  object 
 4   description                       26580 non-null  object 
 5   experiences_offered               26830 non-null  object 
 6   neighborhood_overview             16010 non-null  object 
 7   notes                             10740 non-null  object 
 8   transit                           16719 non-null  object 
 9   access                            15465 non-null  object 
 10  interaction                       14972 non-null  object 
 11  house_rules                       16051 non-null  object 
 12  host

In [775]:
# Checked for NULL, NA, and Duplicated row for the target variable 

print(f"Number of NA cells: {trainDF['price'].isna().sum()}")
print(f"Number of NULL cells: {trainDF['price'].isnull().sum()}")
print(f"Number of duplicated cells: {trainDF['price'].duplicated().sum()}") # keep duplicated rows because some Airbnb listings could have the same price

start_sample_count = trainDF.shape[0]
print(f"Number of samples: {start_sample_count}")

Number of NA cells: 0
Number of NULL cells: 0
Number of duplicated cells: 26301
Number of samples: 26830


In [776]:
# Inspect features for duplicates

# Turned off viewing limit
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

print(f"Number of duplicated rows: {trainDF.duplicated().sum()}")


Number of duplicated rows: 0


In [777]:
# Inspect features for NULL/ NA values
print(trainDF.isnull().sum()) # count of na values

id                                      0
name                                    7
summary                              1009
space                                8347
description                           250
experiences_offered                     0
neighborhood_overview               10820
notes                               16090
transit                             10111
access                              11365
interaction                         11858
house_rules                         10779
host_id                                 0
host_name                               4
host_since                              4
host_location                          87
host_about                          10497
host_response_time                  12998
host_response_rate                  12998
host_acceptance_rate                26830
host_is_superhost                       4
host_neighbourhood                   3774
host_listings_count                     4
host_verifications                

In [778]:
# Express NA values as percentages
((trainDF.isna().sum()/ len(trainDF)) * 100).round(2)

id                                    0.00
name                                  0.03
summary                               3.76
space                                31.11
description                           0.93
experiences_offered                   0.00
neighborhood_overview                40.33
notes                                59.97
transit                              37.69
access                               42.36
interaction                          44.20
house_rules                          40.18
host_id                               0.00
host_name                             0.01
host_since                            0.01
host_location                         0.32
host_about                           39.12
host_response_time                   48.45
host_response_rate                   48.45
host_acceptance_rate                100.00
host_is_superhost                     0.01
host_neighbourhood                   14.07
host_listings_count                   0.01
host_verifi

## Removed N/A values

In [779]:
# Identified that "square_feet" and "host_acceptance_rate" are almost completely NA (98.96% and 100% NA)
# Dropped all features that are over 50% NA ("Notes")
# Dropped "experience_offered" as all cells are listed with "None" instead of NA value

trainDF = trainDF.drop(columns = ["experiences_offered","square_feet", "host_acceptance_rate", "notes"], axis=1)
trainDF.head()

,id,name,summary,space,description,neighborhood_overview,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,15156865,"East 12th st, Lux Studio in Greenwich Village**",A 14-story luxury doorman building at the cent...,NaN,A 14-story luxury doorman building at the cent...,NaN,NaN,NaN,NaN,- Building management requires each new guest ...,22541573,Ken,2014-10-14,"New York, New York, United States","I work for Furnished Quarters, the largest pro...",within an hour,99%,t,Upper East Side,155.0,"['email', 'phone', 'reviews', 'jumio', 'kba', ...",t,t,East Village,Manhattan,New York,NY,10003.0,New York,US,United States,Serviced apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,G...",199.0,1,$0.00,30,1125,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f,flexible,f,f,98,NaN
1,14744912,"sunny airy bohemian rm, private bath! hip 'shwick",A full private bathroom makes you very comfort...,"Large square room, full private bathroom in ha...",A full private bathroom makes you very comfort...,"Interesting street art, people watching, aweso...",We're less than one block away from the L trai...,coin-op laundry room in basement.,I will be available in person or online if you...,"For any questions you must contact us, do not ...",948164,Theodora,2011-08-09,"New York, New York, United States","I am a young professional in Brooklyn, NY. I h...",NaN,NaN,f,NaN,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bushwick,Brooklyn,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,t,f,strict_14_with_grace_period,f,f,1,0.15
2,20280692,"2 bedroom New York, Queens",Quiet 2 bedroom apartment minutes from Astoria...,NaN,Quiet 2 bedroom apartment minutes from Astoria...,NaN,NaN,NaN,NaN,NaN,46407533,Alba,2015-10-12,"New York, New York, United States",NaN,within an hour,100%,f,NaN,1.0,"['email', 'phone', 'reviews']",t,f,Ditmars Steinway,Queens,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,t,f,flexible,f,f,1,0.14
3,3716193,Beautiful 2 Bedroom Townhouse,"Sunny, spacious, beautifully renovated 2 bedro...","Sunny, spacious, beautifully renovated 2 bedro...","Sunny, spacious, beautifully renovated 2 bedro...","Beautiful brownstones, strong community, block...",The A/C train on Nostrand Avenue is 7 minutes ...,"For your cooking needs, the kitchen is fully e...",Depends on the guest's preference but more tha...,"Please don't order pay-per-view movies, we hav...",18970667,"Erin, Avi, Kaleb & Shiloh",2014-07-27,"New York, New York, United States",We are a fun loving couple living in Bed-stuy ...,within an hour,100%,f,Bedford-Stuyvesant,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning..."

In [780]:
# Dropped columns that include textual summaries that vary depending on row (removed unstructured data with varying context)
# These columns include: "name", "summary", "space", "description", "neighborhood", "transit", "access", "interaction", "house_rules", "host_name", "host_about"
trainDF = trainDF.drop(columns=["name", "summary", "space", "description", "neighborhood_overview", "transit", "access", "interaction", "house_rules", "host_name", "host_about"], axis=1)
trainDF.head()

,id,host_id,host_since,host_location,host_response_time,host_response_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,15156865,22541573,2014-10-14,"New York, New York, United States",within an hour,99%,t,Upper East Side,155.0,"['email', 'phone', 'reviews', 'jumio', 'kba', ...",t,t,East Village,Manhattan,New York,NY,10003.0,New York,US,United States,Serviced apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,G...",199.0,1,$0.00,30,1125,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f,flexible,f,f,98,NaN
1,14744912,948164,2011-08-09,"New York, New York, United States",NaN,NaN,f,NaN,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bushwick,Brooklyn,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,t,f,strict_14_with_grace_period,f,f,1,0.15
2,20280692,46407533,2015-10-12,"New York, New York, United States",within an hour,100%,f,NaN,1.0,"['email', 'phone', 'reviews']",t,f,Ditmars Steinway,Queens,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,t,f,flexible,f,f,1,0.14
3,3716193,18970667,2014-07-27,"New York, New York, United States",within an hour,100%,f,Bedford-Stuyvesant,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,t,f,strict_14_with_grace_period,f,f,1,2.92
4,22410246,54586794,2016-01-14,"Paris, Île-de-France, France",NaN,NaN,f,Upper East Side,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Upper East Side,Manhattan,New York,NY,10022,New York,US,United States,Apartment,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,Gym,Elevat...",250.0,1,$0.00,2,7,1,2017-12-31,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,f,flexible,f,f,1,0.10


In [781]:
# Dropped features that have little to no impact of prediction: "id", "host_id", "is_business_travel_ready"(all f), host_location, host_neighbourhood
# Removed host_location b/c the same data is repeated in other features [city, state, zipcode]
# Removed host_neighbourhood b/c it is a duplicated feature with neighbourhood_group_cleansed with little negligable differences
trainDF = trainDF.drop(columns=["id", "host_id", "is_business_travel_ready", "host_location", "host_neighbourhood"], axis=1)
trainDF.head()

,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,2014-10-14,within an hour,99%,t,155.0,"['email', 'phone', 'reviews', 'jumio', 'kba', ...",t,t,East Village,Manhattan,New York,NY,10003.0,New York,US,United States,Serviced apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,G...",199.0,1,$0.00,30,1125,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,f,f,98,NaN
1,2011-08-09,NaN,NaN,f,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bushwick,Brooklyn,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,t,strict_14_with_grace_period,f,f,1,0.15
2,2015-10-12,within an hour,100%,f,1.0,"['email', 'phone', 'reviews']",t,f,Ditmars Steinway,Queens,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,t,flexible,f,f,1,0.14
3,2014-07-27,within an hour,100%,f,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,t,strict_14_with_grace_period,f,f,1,2.92
4,2016-01-14,NaN,NaN,f,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,Upper East Side,Manhattan,New York,NY,10022,New York,US,United States,Apartment,Entire home/apt,4,2.0,2.0,2.0,Real Bed,"{TV,Wifi,""Air conditioning"",Kitchen,Gym,Elevat...",250.0,1,$0.00,2,7,1,2017-12-31,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,f,f,1,0.10


In [782]:
# [host_response_rate] 

# Convert features from precentages (strings) to floats
trainDF["host_response_rate"] = trainDF["host_response_rate"].str.rstrip("%").astype(float)

median_response_rate = trainDF.loc[trainDF["host_response_rate"] < 100, "host_response_rate"].median() # since the distribution heavily-right skewed, median is calculated with values less than 100
trainDF["missing_host_response_rate"] = trainDF["host_response_rate"].isna().astype(int) # created another column to account for missing host_response_rates values (1 for present, 0 for missing)

# Replaced "host_response_rate" NA values with median value
trainDF["host_response_rate"] = trainDF["host_response_rate"].fillna(median_response_rate)


# "missing_host_response_rate" highlights rows that were previously NA/ NULL
trainDF[["host_response_rate", "missing_host_response_rate"]].head(5)

,host_response_rate,missing_host_response_rate
0,99.0,0
1,86.0,1
2,100.0,0
3,100.0,0
4,86.0,1


In [783]:
# [host_response_time]

# Created "missing_host_response_time" to account for missing host_response_rates values (1 for present, 0 for missing)
trainDF["missing_host_response_time"] = trainDF["host_response_time"].isna().astype(int)
display(trainDF[["missing_host_response_time"]].value_counts())

# Add a "missing" cell for NA values in "host_response_time"
trainDF["host_response_time"] = trainDF["host_response_time"].fillna("missing")

# Encoded values using Ordinal encoder
categories = [["missing", "within an hour", "within a few hours", "within a day", "a few days or more"]]
oc = OrdinalEncoder(categories=categories)
trainDF["host_response_time"] = oc.fit_transform(trainDF[["host_response_time"]])

trainDF["host_response_time"].value_counts()


missing_host_response_time
0                             13832
1                             12998
Name: count, dtype: int64

host_response_time
0.0    12998
1.0     8224
2.0     2967
3.0     2210
4.0      431
Name: count, dtype: int64

In [784]:
# Drop NA values from the rest of the dataset
trainDF= trainDF.dropna()
end_sample_count = trainDF.shape[0]
print(f"Percent of dataset retained after data cleaning: {(round(end_sample_count/start_sample_count, 2)*100)}%")

Percent of dataset retained after data cleaning: 75.0%


## Data Preprocessing

In [785]:
# Encoded all binary features
# f = 0.0, t = 1.0

binary_features = ["host_is_superhost", "host_has_profile_pic", "host_identity_verified", "instant_bookable", "require_guest_profile_picture","require_guest_phone_verification"]
encoding = {"f": 0.0, "t": 1.0}

for feature in binary_features:
    trainDF[feature] = trainDF[feature].replace({"f": 0.0, "t": 1.0})

In [786]:
# [host_response_rate]

# Use Min Max Scaler to normalize data (0-1)
min_max_scaler = MinMaxScaler()
trainDF[["host_response_rate"]] = min_max_scaler.fit_transform(trainDF[["host_response_rate"]])
trainDF[["host_response_rate"]].head()

,host_response_rate
1,0.86
2,1.00
3,1.00
5,1.00
6,1.00


In [787]:
# [bed_type] 

# Used One Hot Encoding to encode nominal data
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(trainDF[["bed_type"]])

# Concat data together with the main dataset
encoded_data = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["bed_type"]), index=trainDF.index)
trainDF = pd.concat([trainDF, encoded_data], axis=1)

# Remove the original bed_type feature
trainDF = trainDF.drop(columns=["bed_type"], axis=1)
trainDF.head()

,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,missing_host_response_rate,missing_host_response_time,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed
1,2011-08-09,0.0,0.86,0.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1.0,1.0,Bushwick,Brooklyn,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,strict_14_with_grace_period,0.0,0.0,1,0.15,1,1,0.0,0.0,0.0,0.0,1.0
2,2015-10-12,1.0,1.00,0.0,1.0,"['email', 'phone', 'reviews']",1.0,0.0,Ditmars Steinway,Queens,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,1.0,flexible,0.0,0.0,1,0.14,0,0,0.0,0.0,0.0,0.0,1.0
3,2014-07-27,1.0,1.00,0.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1.0,1.0,Bedford-Stuyvesant,Brooklyn,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,1.0,strict_14_with_grace_period,0.0,0.0,1,2.92,0,0,0.0,0.0,0.0,0.0,1.0
5,2015-07-11,1.0,1.00,0.0,1.0,"['email', 'phone', 'reviews']",1.0,0.0,Kips Bay,Manhattan,New York,NY,10016,New York,US,United States,Apartment,Entire home/apt,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",200.0,2,$50.00,2,1125,75,2015-09-21,2018-10-26,95.0,10.0,9.0,10.0,10.0,10.0,9.0,0.0,strict_14_with_grace_period,0.0,0.0,1,1.97,0,0,0.0,0.0,0.0,0.0,1.0
6,2017-05-22,1.0,1.00,0.0,1.0,"['phone', 'reviews']",1.0,0.0,Astoria,Queens,Queens,NY,11103.0,New York,US,United States,Apartment,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",Kitchen,""Free street ...",60.0,1,$10.00,1,7,95,2017-06-11,2018-10-17,94.0,10.0,9.0,10.0,10.0,10.0,10.0,1.0,flexible,0.0,0.0,1,5.57,0,0,0.0,0.0,0.0,0.0,1.0


In [788]:
# [neighbourhood_group_cleansed] 
# Did the same process as bed_type

# Used One Hot Encoding to encode nominal data
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(trainDF[["neighbourhood_group_cleansed"]])

# Concat data together with the main dataset
encoded_data = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["neighbourhood_group_cleansed"]), index=trainDF.index)
trainDF = pd.concat([trainDF, encoded_data], axis=1)

# Remove the original feature
trainDF = trainDF.drop(columns=["neighbourhood_group_cleansed"], axis=1)
trainDF.head()

,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,missing_host_response_rate,missing_host_response_time,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,neighbourhood_group_cleansed_Bronx,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island
1,2011-08-09,0.0,0.86,0.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1.0,1.0,Bushwick,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,strict_14_with_grace_period,0.0,0.0,1,0.15,1,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,2015-10-12,1.0,1.00,0.0,1.0,"['email', 'phone', 'reviews']",1.0,0.0,Ditmars Steinway,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,1.0,flexible,0.0,0.0,1,0.14,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,2014-07-27,1.0,1.00,0.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",1.0,1.0,Bedford-Stuyvesant,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,1.0,strict_14_with_grace_period,0.0,0.0,1,2.92,0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
5,2015-07-11,1.0,1.00,0.0,1.0,"['email', 'phone', 'reviews']",1.0,0.0,Kips Bay,New York,NY,10016,New York,US,United States,Apartment,Entire home/apt,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",200.0,2,$50.00,2,1125,75,2015-09-21,2018-10-26,95.0,10.0,9.0,10.0,10.0,10.0,9.0,0.0,strict_14_with_grace_period,0.0,0.0,1,1.97,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
6,2017-05-22,1.0,1.00,0.0,1.0,"['phone', 'reviews']",1.0,0.0,Astoria,Queens,NY,11103.0,New York,US,United States,Apartment,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",Kitchen,""Free street ...",60.0,1,$10.00,1,7,95,2017-06-11,2018-10-17,94.0,10.0,9.0,10.0,10.0,10.0,10.0,1.0,flexible,0.0,0.0,1,5.57,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [789]:
# [host_verifications]

# Create a set to hold all unique values
unique_value = set()

# Collect all unique variables in host_verifications
for verification in trainDF["host_verifications"]:
    value = ast.literal_eval(verification)
    unique_value.update(value)

# Apply One Hot Encoding by creating a new column for each unique value and populating based on host_verifications
for value in unique_value:
    trainDF[f"verified_{value}"] = trainDF["host_verifications"].apply(lambda x: float(verification in ast.literal_eval(x)) if pd.notna(x) else 0)

# Removed host_verifications
trainDF = trainDF.drop(columns=["host_verifications"], axis=1)
trainDF.head()

,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,missing_host_response_rate,missing_host_response_time,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,neighbourhood_group_cleansed_Bronx,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,verified_offline_government_id,verified_identity_manual,verified_facebook,verified_government_id,verified_jumio,verified_sent_id,verified_reviews,verified_selfie,verified_phone,verified_manual_offline,verified_sesame_offline,verified_sesame,verified_kba,verified_email,verified_zhima_selfie,verified_google,verified_work_email,verified_weibo,verified_manual_online
1,2011-08-09,0.0,0.86,0.0,1.0,1.0,1.0,Bushwick,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,strict_14_with_grace_period,0.0,0.0,1,0.15,1,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-10-12,1.0,1.00,0.0,1.0,1.0,0.0,Ditmars Steinway,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,1.0,flexible,0.0,0.0,1,0.14,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014-07-27,1.0,1.00,0.0,1.0,1.0,1.0,Bedford-Stuyvesant,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,1.0,strict_14_with_grace_period,0.0,0.0,1,2.92,0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2015-07-11,1.0,1.00,0.0,1.0,1.0,0.0,Kips Bay,New York,NY,10016,New York,US,United States,Apartment,Entire home/apt,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",200.0,2,$50.00,2,1125,75,2015-09-21,2018-10-26,95.0,10.0,9.0,10.0,10.0,10.0,9.0,0.0,strict_14_with_grace_period,0.0,0.0,1,1.97,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2017-05-22,1.0,1.00,0.0,1.0,1.0,0.0,Astoria,Queens,NY,11103.0,New York,US,United States,Apartment,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",Kitchen,""Free street ...",60.0,1,$10.00,1,7,95,2017-06-11,2018-10-17,94.0,10.0,9.0,10.0,10.0,10.0,10.0,1.0,flexible,0.0,0.0,1,5.57,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [790]:
# [amenities]

In [ ]:
# ["cancellation_policy"]
# Ordinal encoded cancellation_policy feature

# Specify categories from least to most strict
cat_order = ["flexible", "moderate", "strict", "strict_14_with_grace_period", "super_strict_30", "super_strict_60"]
oe = OrdinalEncoder(categories=[cat_order])

# Replace cancellation_policy values with encoded values
trainDF["cancellation_policy"] = oe.fit_transform(trainDF[["cancellation_policy"]])
trainDF["cancellation_policy"].value_counts()


cancellation_policy
3.0    10146
1.0     5352
0.0     4700
5.0       10
4.0        8
2.0        1
Name: count, dtype: int64

In [ ]:
# [reviews_per_month]

# Standardized reviews_per_month with standard scaler b/c most values are already less than 1 with few outliers
scaler = StandardScaler()
trainDF["reviews_per_month"] = scaler.fit_transform(trainDF[["reviews_per_month"]])
trainDF["reviews_per_month"].head()

1   -0.782088
2   -0.788031
3    0.864265
5    0.299631
6    2.439295
Name: reviews_per_month, dtype: float64

In [802]:
trainDF.head()

,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,city,state,zipcode,market,country_code,country,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,guests_included,extra_people,minimum_nights,maximum_nights,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,missing_host_response_rate,missing_host_response_time,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,neighbourhood_group_cleansed_Bronx,neighbourhood_group_cleansed_Brooklyn,neighbourhood_group_cleansed_Manhattan,neighbourhood_group_cleansed_Queens,neighbourhood_group_cleansed_Staten Island,verified_offline_government_id,verified_identity_manual,verified_facebook,verified_government_id,verified_jumio,verified_sent_id,verified_reviews,verified_selfie,verified_phone,verified_manual_offline,verified_sesame_offline,verified_sesame,verified_kba,verified_email,verified_zhima_selfie,verified_google,verified_work_email,verified_weibo,verified_manual_online
1,2011-08-09,0.0,0.86,0.0,1.0,1.0,1.0,Bushwick,Brooklyn,NY,11237.0,New York,US,United States,Apartment,Private room,3,1.0,1.0,2.0,"{TV,Internet,Wifi,""Air conditioning"",Kitchen,""...",80.0,1,$10.00,2,300,4,2016-09-06,2018-04-30,95.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,3.0,0.0,0.0,1,-0.782088,1,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015-10-12,1.0,1.00,0.0,1.0,1.0,0.0,Ditmars Steinway,Queens,NY,11105.0,New York,US,United States,Apartment,Entire home/apt,4,1.0,2.0,3.0,"{TV,""Cable TV"",Wifi,""Air conditioning"",Kitchen...",160.0,1,$0.00,5,1125,2,2017-09-03,2018-09-02,100.0,10.0,10.0,8.0,9.0,9.0,10.0,1.0,0.0,0.0,0.0,1,-0.788031,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014-07-27,1.0,1.00,0.0,1.0,1.0,1.0,Bedford-Stuyvesant,Brooklyn,NY,11216,New York,US,United States,Apartment,Entire home/apt,6,1.0,2.0,3.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,2,$15.00,3,400,151,2014-08-09,2018-10-21,93.0,10.0,9.0,10.0,10.0,9.0,9.0,1.0,3.0,0.0,0.0,1,0.864265,0,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2015-07-11,1.0,1.00,0.0,1.0,1.0,0.0,Kips Bay,New York,NY,10016,New York,US,United States,Apartment,Entire home/apt,2,1.0,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",200.0,2,$50.00,2,1125,75,2015-09-21,2018-10-26,95.0,10.0,9.0,10.0,10.0,10.0,9.0,0.0,3.0,0.0,0.0,1,0.299631,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2017-05-22,1.0,1.00,0.0,1.0,1.0,0.0,Astoria,Queens,NY,11103.0,New York,US,United States,Apartment,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",Kitchen,""Free street ...",60.0,1,$10.00,1,7,95,2017-06-11,2018-10-17,94.0,10.0,9.0,10.0,10.0,10.0,10.0,1.0,0.0,0.0,0.0,1,2.439295,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [806]:
trainDF["city"].value_counts()

city
New York                                  9104
Brooklyn                                  7921
Queens                                    1682
Bronx                                      340
Brooklyn                                   325
Staten Island                              133
Astoria                                     69
Long Island City                            45
New York                                    30
Flushing                                    29
Flushing                                    27
Astoria                                     23
Woodside                                    22
Bronx                                       21
Elmhurst                                    21
Jamaica                                     16
Ridgewood                                   16
Elmhurst                                    13
Ridgewood                                   12
brooklyn                                    12
Queens                                      11
flushing

In [ ]:
trainDF["city"] = trainDF["city"].str.lower().str.strip()

trainDF["city"].value_counts()

city
new york                                  9147
brooklyn                                  8267
queens                                    1694
bronx                                      364
staten island                              141
astoria                                     95
flushing                                    67
long island city                            55
elmhurst                                    34
woodside                                    30
ridgewood                                   28
jamaica                                     25
jackson heights                             19
new york city                               12
east elmhurst                               10
rockaway beach                              10
sunnyside                                    9
nyc                                          8
forest hills                                 8
arverne                                      8
corona                                       7
clinton 

In [ ]:
# only account for the top 50, set the rest as other

In [805]:
trainDF["state"].value_counts()

state
NY    20214
NJ        1
Ny        1
ny        1
Name: count, dtype: int64

Data Quality Issues

In terms of data quality issues, this dataset has many missing/ NA values as well as context dependent data. For example, features such as "summary" and "description" are textual data, specific to each listing and can't be used as a comparison metric.

# 1. Model Training (60 points)

Train at least 3 models. You can evaluate more than 3 models and apply your own cross-validation approach to choose the top three models to submit to Kaggle. For each model, include a short write up on each model (desc of selected model, evaluation approach applied, how did you select hyperparameters).  

In [ ]:
# Model 1 - Linear Regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression



[INSERT WRITE-UP HERE]

In [ ]:
# Model 2

[INSERT WRITE-UP HERE]

In [ ]:
# Model 3

[INSERT WRITE-UP HERE]

# 2. Plotting (10 points)

In [ ]:
# Plot results here
# [INSERT CODE HERE]

[INSERT WRITE-UP HERE]

# 3. Discussion (10 points)

# include a short discussion about results and approach.

Note, the rest of the points are based on Kaggle submission and ranking.

# References / Disclosure

[List the sources you used to complete this assignment here]